In [1]:


# %pip install ultralytics opencv-python pandas matplotlib tqdm

import os
import cv2
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm

# Configura las rutas
input_dir = "videos"       # 📂 carpeta con los videos a analizar
output_dir = "resultados"  # 📂 carpeta donde se guardarán los resultados
os.makedirs(output_dir, exist_ok=True)

# Parámetros del modelo
model_name = "yolo11s.pt"  # Puedes usar yolo11n.pt (rápido) o yolo11m/l/x.pt (más preciso)
conf_thres = 0.25          # Umbral de confianza
device = "cpu"             # Usa "cuda" si tienes GPU
use_tracking = True        # Activa tracking para IDs únicos

# Cargar modelo YOLOv11
model = YOLO(model_name)
print("✅ Modelo cargado correctamente:", model_name)

✅ Modelo cargado correctamente: yolo11s.pt


In [ ]:

def procesar_video(video_path, model, output_dir,
                   conf=0.25, device="cpu",
                   use_tracking=True, save_annotated=False,
                   frame_skip=28,  # 🔹 procesa 1 de cada 5 frames
                   max_frames=None  # 🔹 límite opcional de frames (None = todos)
                   ):
    """
    Procesa un video largo con muestreo de frames y tracking opcional.
    Guarda métricas resumidas sin saturar memoria.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ No se pudo abrir: {video_path}")
        return None

    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    base_name = os.path.splitext(os.path.basename(video_path))[0]
    out_path = os.path.join(output_dir, f"{base_name}_annotated-sg2.mp4") if save_annotated else None

    writer = None
    if save_annotated:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    print(f"🎥 Procesando: {base_name} ({total_frames} frames aprox.)")
    print(f"⚙️  Procesando 1 de cada {frame_skip} frames para optimizar rendimiento")

    frame_id = 0
    total_personas = 0
    max_concurrent = 0
    unique_ids = set()
    processed_frames = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_id += 1

        # Saltar frames según el muestreo
        if frame_id % frame_skip != 0:
            continue

        # Limitar cantidad total procesada (útil para pruebas)
        if max_frames and processed_frames >= max_frames:
            break

        results = model.predict(frame, conf=conf, device=device, classes=[0], verbose=False)

        count = 0
        if results and len(results[0].boxes) > 0:
            for b in results[0].boxes:
                if int(b.cls[0]) == 0:
                    count += 1
                    if use_tracking and hasattr(b, "id") and b.id is not None and len(b.id) > 0:
                        unique_ids.add(int(b.id[0]))

        total_personas += count
        max_concurrent = max(max_concurrent, count)
        processed_frames += 1

        # Guardar video anotado opcionalmente
        if save_annotated:
            annotated = results[0].plot()
            writer.write(annotated)

        # Liberar memoria del frame
        del results, frame

    cap.release()
    if writer:
        writer.release()

    return {
        "video": base_name,
        "frames_totales": total_frames,
        "frames_procesados": processed_frames,
        "fps": round(fps, 2),
        "muestreo": frame_skip,
        "sum_persons_over_frames": total_personas,
        "max_concurrent_persons": max_concurrent,
        "unique_ids": len(unique_ids) if use_tracking else None,
        "annotated_path": out_path if save_annotated else None
    }


In [6]:
# ============================================
# 3️⃣ Procesar todos los videos de una carpeta
# ============================================

def listar_videos(carpeta):
    exts = (".mp4", ".avi", ".mov", ".mkv", ".webm")
    return [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith(exts)]

videos = listar_videos(input_dir)
if not videos:
    print("⚠️ No se encontraron videos en la carpeta:", input_dir)
else:
    print(f"Se encontraron {len(videos)} videos para analizar.")

resultados = []
for v in videos:
    r = procesar_video(v, model, output_dir, conf_thres, device, use_tracking, save_annotated=True)
    if r:
        resultados.append(r)


Se encontraron 1 videos para analizar.
🎥 Procesando: video (9005 frames aprox.)
⚙️  Procesando 1 de cada 28 frames para optimizar rendimiento


In [7]:
# ============================================
# 4️⃣ Exportar resultados a CSV y mostrar resumen
# ============================================

df = pd.DataFrame(resultados)
csv_path = os.path.join(output_dir, "resumen_conteo_personas_muestra-s2.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

print("\n✅ Análisis completado.")
print("📄 Resultados guardados en:", csv_path)
display(df)



✅ Análisis completado.
📄 Resultados guardados en: resultados/resumen_conteo_personas_muestra-s2.csv


,video,frames_totales,frames_procesados,fps,muestreo,sum_persons_over_frames,max_concurrent_persons,unique_ids,annotated_path
0,video,9005,321,30.0,28,3640,16,0,resultados/video_annotated-sg2.mp4


In [ ]:
# ============================================
# 5️⃣ Visualización rápida (opcional)
# ============================================

import matplotlib.pyplot as plt

if not df.empty:
    plt.figure(figsize=(10,5))
    plt.bar(df["video"], df["max_concurrent_persons"], color="skyblue")
    plt.title("Pico simultáneo de personas por video")
    plt.ylabel("Número máximo de personas")
    plt.xlabel("Video")
    plt.xticks(rotation=45, ha='right')
    plt.grid(alpha=0.3)
    plt.show()
else:
    print("⚠️ No hay datos para graficar.")
